In [1]:
using PyPlot
include("saltsolver.jl")

/Users/davidliu/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


smallest_stability_eigs (generic function with 1 method)

In [18]:
nsym = 7
ℓ = 2
Lcav = 1.37

wantcnv = true
if wantcnv
    ɛrpiece = [1.0, 1.1, 1.1]
    ɛipiece = [0.3, 0.25, 0.25]
    Fpiece = [1.1, 0.9, 0.9]
else 
    ɛrpiece = [1.0, 1.2, 1.1]
    ɛipiece = [0.3, 0.25, 0.32]
    Fpiece = [1.1, 0.9, 0.95]
end
    
ɛr = cn_profile(ɛrpiece, nsym)
ɛi = cn_profile(ɛipiece, nsym)
ɛ = ɛr + im*ɛi
F = cn_profile(Fpiece, nsym)

ismirror(v) = (v == ring_flip(v))
iscnv = (ismirror(ɛ) && ismirror(F))
assert(wantcnv == iscnv)

N = length(F)
h = Lcav / N
laplacian!(J) = periodic!(J, h)
Eguess, ωguess = ring_eigenpair(N, ℓ, h)

# purposely make it far from gain center to 
# see low-Q effects
ωa = round(ωguess) + 1
γ⟂ = 0.7
γpar = 0.1
las = Laser(ɛ, F, ωa, γ⟂);

In [19]:
Dguess = 1.5* mean(ɛipiece)
md = Mode(copy(Eguess), ωguess, 0.0)
Dt = threshold!(laplacian!, md, las, (Dguess, Dguess*1.05))

println("threshold search slightly breaks pure chirality")
println("use projection operator to get it back")
md = Mode(project_chiral(md.E, nsym, ℓ), md.ω, md.c²)
println("chiral-projected mode should automatically be solution")
solve!(laplacian!, md, las, Dt, isprint=true)
println("ℓ eigenvalue and residual:\n", get_ell(md.E, nsym))

fmd = mode_flip(md)
if iscnv
    println("Cnv, so flipped solved immediate")    
else
    println("Cn only, so flipped solve not immediate")
end
solve!(laplacian!, fmd, las, Dt, isprint=true)
if !iscnv
    fmd = Mode(project_chiral(fmd.E, nsym, -ℓ), fmd.ω, fmd.c²)
    println("reprojecting fmd to fix slightly broken chiral symmetry")
    println("reprojected fmd should immediately solve Newton")
    solve!(laplacian!, fmd, las, Dt, isprint=true)
end
println("ℓ eigenvalue and residual:\n", get_ell(fmd.E, nsym))

println("\ncheck that degenerate and at threshold")
println("ω₊ - ω₋ = ", md.ω - fmd.ω)
println("|c₊²|+|c₋²| = ", abs(md.c²) + abs(fmd.c²))
Es = (md.E, fmd.E)
ωt = md.ω
H, I, J, K, GD = overlap_integrals(Es, ωt, Dt, las, nsym, ℓ, Lcav);

threshold search slightly breaks pure chirality
use projection operator to get it back
chiral-projected mode should automatically be solution
|f| = 1.497946573415776e-9
ℓ eigenvalue and residual:
(2.0000000000000004,1.9621883384912796e-16)
Cnv, so flipped solved immediate
|f| = 1.497973526832406e-9
ℓ eigenvalue and residual:
(-2.0000000000000004,2.89503656367659e-16)

check that degenerate and at threshold
ω₊ - ω₋ = 0.0
|c₊²|+|c₋²| = 1.7706439429662562e-10


In [20]:
# CIRCULATING MODES
ω1s = [0.0, 0.0]
a²ths = [0.0, 0.0]
for m in (1, 2)
    ω1s[m] = -imag(GD / I[m]) / imag(H / I[m])
    a²ths[m] = real((ω1s[m]*H+GD) / I[m])
end

m = 2
d = 0.0001
Dlasing = Dt*(1+d)
a² = a²ths[m]
ω1 = ω1s[m]
md = Mode(copy(Es[m]), ωt+ω1*d, d * a²)
solve!(laplacian!, md, las, Dlasing, isprint=true)
Esalt = √md.c² * md.E
ωsalt = md.ω

println("expected a² = ", a²)
println("actual   a² = ", md.c² / d)
println("expected ω1 = ", ω1)
println("actual   ω1 = ", (md.ω-ωt)/d)

println("\n theoretical stability eigenvalues:")
imjh = imag(J[m]/H)
term = (abs(K[m])^2 - abs(J[m])^2) / abs(H)^2
other_val(z) = a²*(imjh + z*√(0.0im + imjh^2 + term))
println(0.0)
println(other_val(1))
println(other_val(-1))
println(2a² * imag(I[m] / H))

println("\nnumerical stability eigenvalues")
Λsmallest = smallest_stability_eigs(laplacian!, Esalt, ωsalt, Dlasing, γpar, las.ɛ)
for λ in Λsmallest
    println(λ / d)
end

|f| = 0.00032476972469017054
|f| = 3.1451120133688914e-13
expected a² = 1.0187376551272047
actual   a² = 1.0187389560772806
expected ω1 = 1.467122116840567e-5
actual   ω1 = 1.467196142357352e-5

 theoretical stability eigenvalues:
0.0
-0.633074236399256 + 0.13824705857396083im
-0.633074236399256 - 0.13824705857396083im
-1.266148472798512

numerical stability eigenvalues
1.3481174159880266e-9 + 0.0im
-0.6334367796214274 + 0.13839710034504804im
-0.6334367796214274 - 0.13839710034504804im
-1.2677775269641791 + 0.0im


In [21]:
# get standing mode allowed phases
θroots, ω1roots, a²standing = standing_roots_test(H, I, J, K, GD, iscnv, nsym, ℓ)

Cnv but not n=4|ℓ|; zeros everywhere
|K₊|+|K₋| = 3.4604579259646835e-17
if K=0, then T independent of z
and ω1₊ = ω1₋ satisfied for all z because
Cnv -> I₊ = I₋)
outputting random phase angle
standing lasing mode E = |a₊|E₊ exp(iθ)|a₋|E₋
θ1 = 0.20683657821797585 × π/2
ω1[1] = 1.4671221168474631e-5
  |a₊|²[1] = 0.33957921837573507
  |a₋|²[1] = 0.33957921837573496
|a₊|²=|a₋|² because mirror symmetry


([0.32489813731162176],[1.4671221168474631e-5],[(0.33957921837573507,0.33957921837573496)])

In [22]:
mroot = 1
d = 0.0001
Dlasing = Dt * (1+d)
a²s = a²standing[mroot]
θroot = θroots[mroot]
Esalt_guess = √a²s[1] * Es[1] + exp(im*θroot) * √a²s[2] * Es[2]
ωsalt_guess = ωt + d * ω1roots[mroot]
md = Mode(copy(Esalt_guess), ωsalt_guess, 0.0)
c²guess = d * norm(Esalt_guess)^2 / norm(md.E)^2
md.c² = c²guess
solve!(laplacian!, md, las, Dlasing, isprint=true)
coefs = √md.c² * collect(coefficients( md.E, Es))
Esalt = √md.c² * md.E
ωsalt = md.ω
println("expected c² = ", c²guess)
println("actual   c² = ", md.c²)

println("expected ω1 = ", ω1roots[mroot])
println("actual   ω1 = ", (md.ω - ωt)/d)
for m in (1, 2)
    println("coefficient |a_", m, "|^2:")
    println("\texpected: ", a²s[m])
    println("\tactual:   ", abs(coefs[m])^2/d)
end
println("expected θ = ", θroot)
println("actual   θ = ", angle(coefs[2]/coefs[1]))

println("\nnumerical stability eigenvalues")
Λsmallest = smallest_stability_eigs(laplacian!, Esalt, ωsalt, Dlasing, γpar, las.ɛ)
for λ in Λsmallest
    println(λ / d)
end

println("\ntheoretical stability eigenvalues")
z = exp(im*θroot)
a₊a₋ = sqrt(a²s[1]*a²s[2])
a₊², a₋² = a²s 
# be careful to either say 1 + 1 or 1+1 here, not 1 +1 or else
# it will get interpreted as the next column!
XQmP̃ = [2a₋²*K[2]*z^2   a₊a₋*((z-1/z)*(I[2]+J[2]) - 2z*K[2]) ;
        -2a₊a₋/z*K[1]       a₋²*(z^2-1)*I[2] + a₊²*K[1]*(1+z^-2)     ]
XQpP̃ = [2a₊²*I[1]      a₊a₋*((z+1/z)*(I[2]+J[2]) + 2z*K[2]); 
    2a₊a₋*(z*(I[1] + J[1]) + 1/z*K[1])    a₋²*(z^2+1)*I[2] + a₊²*K[1]*(1-z^-2)] 
MatFinal = [ imag(XQpP̃/H)   -real(XQmP̃/H);
            -real(XQpP̃/H)  -imag(XQmP̃/H) ]

Λfinal, ~ = eig(MatFinal)
Λfinal = Λfinal[sortperm(abs(Λfinal))]
for λ in Λfinal
    println(λ)
end

i₊, i₋ = I[1]/H, I[2]/H
j₊, j₋ = J[1]/H, J[2]/H
k₊, k₋ = K[1]/H, K[2]/H
zr, zi = real(z), imag(z)
MatFinalAlternate_first2cols = 
2*[ a₊² * imag(i₊)      a₊a₋*(zr*imag(i₋ + j₋) + imag(z*k₋));
    a₊a₋*imag(z*(i₊ + j₊)+1/z*k₊)     a₋²*zr*imag(z*i₋) + a₊²*real(1/z*k₊)*zi;
    -a₊² * real(i₊)      -a₊a₋*(zr*real(i₋+j₋) + real(z*k₋));
    -a₊a₋*real(z*(i₊+j₊)+1/z*k₊)    -a₋²*zr*real(z*i₋) + a₊²*imag(1/z*k₊)*zi]
MatFinalAlternate_second2cols = 
2*[ -a₋²*real(k₋*z^2)   a₊a₋*(zi*imag(i₋+j₋) + real(z*k₋));
    a₊a₋*real(1/z*k₊)   a₋²*zi*imag(z*i₋) - a₊²*real(1/z*k₊)*zr;
    -a₋²*imag(k₋*z^2)   -a₊a₋*(zi*real(i₋+j₋) - imag(z*k₋));
    a₊a₋*imag(1/z*k₊)   -a₋²*zi*real(z*i₋) - a₊²*imag(1/z*k₊)*zr]
MatFinalAlternate = [MatFinalAlternate_first2cols  MatFinalAlternate_second2cols]
println("\nnorm(MatFinal - MatFinalAlternate) = ", norm(MatFinalAlternate - MatFinal))

# this part commented out because intended for z = ±1 for C4ℓ case
# but must manually phase rotate E₋ such that the θroots are multiples
# of π/2
# if nsym==4abs(ℓ) && !iscnv
#     println("\nC4ℓ, solving 3x3 matrix problem numerically")
#     for sn in (1, -1)
#         println("theoretical eigenvalues for sign = ", sn)
#         Mat3x3 = zeros(3,3)
#         α = √(a₊² + a₋²)
#         Mat3x3[1,1] = a₊² * imag(i₊)
#         Mat3x3[1,2] = sn * a₊a₋ * imag(i₋ + j₋ + k₋)
#         Mat3x3[1,3] = -α * √a₋² * real(k₋)
#         Mat3x3[2,1] = sn * a₊a₋ * imag(i₊ + j₊ + k₊)
#         Mat3x3[2,2] = a₋² * imag(i₋)
#         Mat3x3[2,3] = sn * α * √a₊² * real(k₊)
#         Mat3x3[3,1] = a₊² * √a₋²/α * real(j₊ + k₊)
#         Mat3x3[3,2] = -sn * a₋² * √a₊²/α * real(j₋ + k₋)
#         Mat3x3[3,3] = -a₋² * imag(k₋) - a₊² * imag(k₊)
#         Mat3x3 *= 2
        
#         vals, ~ = eig(Mat3x3)
#         abslt(x, y) = abs(x) < abs(y)
#         sort!(vals, lt=abslt)        
#         for λ in vals
#             println("\t", λ)
#         end
#     end
# end

if nsym==4abs(ℓ) && iscnv
    println("\nclosed-form theoretical predictions for Cnv with n==4ℓ:")
    isrealz = iseven(round(Int, θroot/(π/2)))
    println("standing solution with z = ", isrealz ? "±1" : "±i")
    vals = [0.0, 0.0, 0.0, 0.0]
    vals[1] = 0.0
    
    α = isrealz ? 1 : -1
    # I have no idea what to name this!
    
    vals[2] = 2a₊² * imag(2i₊ + j₊ + α*k₊)
    radicand = imag(k₊ - α * j₊)^2 - α * 8real(k₊)*real(j₊ + α * k₊)    
    vals[3] = -a₊² * imag(j₊ + α * 3k₊) + a₊² * √radicand
    vals[4] = -a₊² * imag(j₊ + α * 3k₊) - a₊² * √radicand
    abslt(x, y) = abs(x) < abs(y)
    sort!(vals, lt=abslt)
    for λ in vals
        println(λ)
    end
elseif iscnv && nsym!=4abs(ℓ)
    println("\nclosed-form theoretical predictions for Cnv with n!=4ℓ:")    
    println("two nonzero eigenvalues")
    vals = [0.0, 0.0]
    ss = [1, -1]
    tr0 = 2imag(a₊² * i₊ + a₋² * i₋)
    det0 = -4a₊a₋^2*(imag(i₊)*imag(j₋) + imag(j₊)*imag(i₋) + imag(j₊)*imag(j₋))
    for m = (1, 2)
        vals[m] = tr0/2 + ss[m] * sqrt((tr0/2)^2 - det0)
    end
    for λ in vals
        println(λ)
    end
end

|f| = 0.0031089233396190312
|f| = 3.6388762276808624e-13
expected c² = 0.00013576639966849725
actual   c² = 0.00013576825260256964
expected ω1 = 1.4671221168474631e-5
actual   ω1 = 1.5033858602464534e-5
coefficient |a_1|^2:
	expected: 0.33957921837573507
	actual:   0.33958303263849055
coefficient |a_2|^2:
	expected: 0.33957921837573496
	actual:   0.33958303263926337
expected θ = 0.32489813731162176
actual   θ = 0.32489814079566803

numerical stability eigenvalues
8.134349861634375e-12 + 0.0im
-2.0330080081148283e-10 + 0.0im
0.42202654110800786 + 0.0im
-1.267764583056757 + 0.0im

theoretical stability eigenvalues
1.9943913570024153e-17
-6.899166110693148e-17
0.4220494909328374
-1.266148472798512

norm(MatFinal - MatFinalAlternate) = 1.2673571756096142e-16

closed-form theoretical predictions for Cnv with n!=4ℓ:
two nonzero eigenvalues
0.4220494909328375
-1.2661484727985122
